In [ ]:
# csrftoken
csrftoken = ''

# ssuz_sessionid
ssuz_sessionid = ''

In [32]:
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

from queue import Queue
import threading
from datetime import datetime
import re

from concurrent.futures import ThreadPoolExecutor

import os
import json


In [31]:
url = [
            # Получение основного id и name группы
            'https://ssuz.vip.edu35.ru/actions/register/lessons_tab/lessons_tab_group_rows',
            # Получение id предмета и подгруппы
            'https://ssuz.vip.edu35.ru/actions/register/lessons_tab/lessons_tab_subject_rows',
            # Получение студентов группы
            'https://ssuz.vip.edu35.ru/actions/register/lessons_tab/lessons_tab_rows',
            # Закрытие занятия
            'https://ssuz.vip.edu35.ru/actions/register/lessons_tab/lessons_tab_close_lesson_action',
            # Открыть занятие
            'https://ssuz.vip.edu35.ru/actions/register/lessons_tab/lessons_tab_open_lesson_action',
            # Выставить явку
            'https://ssuz.vip.edu35.ru/actions/register/lessons_tab/save_lesson_score',
            # Создание поля для оценок
            'https://ssuz.vip.edu35.ru/actions/lesson_work/objectsaveaction',
            # Получение id оценок
            'https://ssuz.vip.edu35.ru/actions/lesson_work/objectrowsaction',
            # Выставление оценок
            'https://ssuz.vip.edu35.ru/actions/register/lessons_tab/save_lesson_score',
            # Запись темы урока
            'https://ssuz.vip.edu35.ru/actions/register/lesson_register/lesson_register_save',
            # Получение название подгрупп
            'https://ssuz.vip.edu35.ru/actions/ssuz.register.actions.Pack/finalmarktypewindowaction',
            # Задать вид оценки
            'https://ssuz.vip.edu35.ru/actions/ssuz.register.actions.Pack/finalmarktypesaveaction',
        ]

cookies = '11'
session = requests.session()

mon = int(datetime.today().strftime('%m'))
if 13 > mon > 8:
    period_id = int(datetime.today().strftime('%y')) + 8 + (int(datetime.today().strftime('%y')) - 23)
else:
    period_id = int(datetime.today().strftime('%y')) + 8 + (int(datetime.today().strftime('%y')) - 23) + 1
# print('rej 58',period_id)
# print(session)

# Папка для сохранения промежуточных данных
OUTPUT_DIR = "intermediate"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Файл для итогового результата
FINAL_FILE = "final_result.txt"


In [ ]:
def head(cookie):
    '''Создание заголовка'''
    head_ = {  # Заголовок запроса
        'Host': 'ssuz.vip.edu35.ru',
        'User-Agent': 'Mozilla/5.0 (Linux; Android 8.0.0; SM-G955U Build/R16NW) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Mobile Safari/537.36 Edg/119.0.0.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
        'Accept-Encoding': 'gzip, deflate, br',
        'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'Cookie': cookie,
        'X-Requested-With': 'XMLHttpRequest',
        'Referer': 'https://ssuz.vip.edu35.ru/auth/login-page',
        'Sec-Ch-Ua': '"Chromium";v="116", "Not)A;Brand";v="24", "Microsoft Edge";v="116"',
        'Cache-Control':'max-age=0',
        'Sec-Fetch-Dest':'document',
        'Sec-Fetch-Mode':'navigate',
        'Sec-Fetch-Site':'cross-site',
        'Sec-Fetch-User':'?1',
        'Upgrade-Insecure-Requests':'1',
        'Sec-Ch-Ua-Mobile': '?0',
        'Sec-Ch-Ua-Platform': '"Windows"',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'X-Xsrftoken': cookie[cookie.find('csrftoken=') + len('csrftoken='):]
    }
    return head_

def set_cookie(cookie):
    global cookies
    print('cookie',cookie)
    '''Установка куки(Для тестов, а то банит XD)'''
    session.get('https://ssuz.vip.edu35.ru', cookies={'Cookie': cookie},verify=False)
    cookies = cookie
    print(cookie)

def login(login, password):
        '''Авторизация'''
        try:
            # cook = requests.utils.dict_from_cookiejar(session.cookies)
            # print("11111111111111111111111111111",cook)
            set_cookie(f'csrf_token_header_name=X-XSRFTOKEN;ssuz_sessionid={password};csrftoken={login}')
            return True
        except KeyError:
            return False

def date_patch(date_f=''):
        """Выставление правильного времени"""
        date_from = date_f
        if date_f == '':
            mon = int(datetime.today().strftime('%m'))
            # date_from = '01.09.2023'# + datetime.today().strftime('%Y')
            if 13 > mon > 8:
                date_from = '01.09.' + datetime.today().strftime('%Y')
            else:
                date_from = '01.01.' + datetime.today().strftime('%Y')
        return date_from
        
def group_rows(prac='', date_from='', date_to=datetime.today().strftime('%d.%m.%Y')):
        '''Получение групп'''
        global cookies

        date_from = date_patch(date_from)
        print(date_from,'date_from')
        data = {
            'slave_mode': '1',
            'empty_item': '1',
            'practical': prac,
            'unit_id': '22',
            'period_id': period_id,
            'date_from': date_from,
            'date_to': date_to,
            'month': '',
            'filter': ''
        }
        # print('rej 139',self.session.post(self.url[0], headers=self.head(), data=data))
        # print(1,head(cookies))
        response = session.post(url[0], headers=head(cookies),verify=False, data=data).json()
        # print(141,response)
        # print(141,response.content)
        return response

def disc_rows( id_group, prac='', date_from=''):
    '''Получение предметов группы'''
    global cookies

    date_from = date_patch(date_from)
    data = {
        'slave_mode': '1',
        'empty_item': '1',
        'practical': prac,
        'unit_id': '22',
        'period_id': period_id,
        'date_from': date_from,
        'date_to': datetime.today().strftime('%d.%m.%Y'),
        'month': '',
        'filter': '',
        'group_id': id_group,
        'subject': '0',
        'subject_gen_pr_id': '0',
        'exam_subject_id': '0',
        'subject_sub_group_obj': '',
    }
    response = session.post(url[1], headers=head(cookies), data=data,verify=False).json()
    return response

def student_rows(id_group, subject_id, date_from='',date_whis=datetime.today().strftime('%d.%m.%Y'), prac=''):
    '''Получение студентов группы'''
    global cookies

    date_from = date_patch(date_from)

    nums = re.findall(r'\d+', subject_id)
    data = {
        'slave_mode': '1',
        'empty_item': '1',
        'practical': prac,
        'unit_id': '22',
        'period_id': period_id,
        'date_from': date_from,
        'date_to': date_whis,
        'month': '',
        'filter': '',
        'group_id': id_group,
        'subject': '0',
        'subject_gen_pr_id': '0',
        'exam_subject_id': '0',
        'subject_sub_group_obj': subject_id,
        'subject_id': nums[0],
    }
    s = session.post(url[2], headers=head(cookies), data=data,verify=False)
    response = s.json()
    # print('rej 189',response)
    return response

def creat_str(name, id_group, subject_id, student_id, id_list):
    '''Создание списка практических и теоретических журналов'''
    s = f"'name': '{name} ', 'id_group': '{id_group}', 'subject_id': '{subject_id}','student_id': '{student_id}', 'id':{id_list}"
    return '\t{' + s + '},\n'

def create_disc_list():
        global group_rows
        '''Создание списков для работы'''
        prac = 'Practice = [\n'
        theo = 'Theory = [\n'
        p_id = 0
        t_id = 0
        for group in group_rows(prac='', date_from=date_patch())['rows']:
            print(group)
            name = group['name'][:group['name'].index(' ')]
            # print(name)
            # Получение предметов группы
            for disc in disc_rows(group['id'], prac='', date_from=date_patch())['rows']:
                # Получение студентов группы
                student = student_rows(group['id'], disc['id'], prac='', date_from=date_patch())
                # print(name)
                # que.put(name)
                if student['total'] <= 5:
                    theo += creat_str(name + '_' + disc['name'], group['id'], disc['id'],student['rows'][0]['student_id'], t_id)
                    prac += creat_str(name + '_' + disc['name'], group['id'], disc['id'],student['rows'][0]['student_id'], p_id)
                    p_id += 1
                    t_id += 1
                elif '(' in disc['name']:
                    prac += creat_str(name + '_' + disc['name'], group['id'], disc['id'],student['rows'][0]['student_id'], p_id)
                    p_id += 1
                else:
                    theo += creat_str(name + '_' + disc['name'], group['id'], disc['id'],student['rows'][0]['student_id'], t_id)
                    t_id += 1
        return [prac + ']\n', theo + ']']      



In [35]:
def create_disc_list2():
    '''Создание списков для работы (многопоточная версия)'''
    prac_header = 'Practice = [\n'
    theo_header = 'Theory = [\n'

    # Получаем список всех групп
    groups = group_rows(prac='', date_from=date_patch())['rows']

    # Параллельно обрабатываем каждую группу в отдельном потоке
    with ThreadPoolExecutor(max_workers=10) as executor:  # Можно изменить max_workers
        results = executor.map(process_group, groups)

    # Собираем результаты из всех потоков
    practice_lines = ''
    theory_lines = ''

    for p, t in results:
        practice_lines += p
        theory_lines += t

    # Формируем финальные строки
    full_practice = prac_header + practice_lines + ']\n'
    full_theory = theo_header + theory_lines + ']\n'

    return [full_practice, full_theory]


def process_group(group):
    '''
    Обработка одной группы в отдельном потоке.
    Возвращает строку для Practice и Theory.
    '''
    prac_str = ''
    theo_str = ''

    # Извлекаем имя группы до первого пробела
    name = group['name'].split(' ')[0] if ' ' in group['name'] else group['name']

    # Получаем предметы группы
    disc_data = disc_rows(group['id'], '',date_patch())
    discs = disc_data.get('rows', [])

    for disc in discs:
        # Получаем студентов по предмету
        student_data = student_rows(group['id'],disc['id'], prac='', date_from=date_patch())
        students = student_data.get('rows', [])

        if not students:
            continue

        student_id = students[0]['student_id']

        # Логика формирования строк
        full_name = f"{name}_{disc['name']}"

        if student_data['total'] <= 5:
            prac_str += creat_str(full_name, group['id'], disc['id'], student_id, 0)
            theo_str += creat_str(full_name, group['id'], disc['id'], student_id, 0)
        elif '(' in disc['name']:
            prac_str += creat_str(full_name, group['id'], disc['id'], student_id, 0)
        else:
            theo_str += creat_str(full_name, group['id'], disc['id'], student_id, 0)

    return prac_str, theo_str


# Пример функции creat_str — замените на свою, если она уже есть
def creat_str(name, group_id, disc_id, student_id, index):
    # Здесь можно добавить форматирование, как тебе нужно
    return f'("{name}", {group_id}, {disc_id}, {student_id}),\n'



# def create_disc_list3():
#     '''Создание списков для работы (многопоточная версия с сохранением в файл)'''
#     prac_header = 'Practice = [\n'
#     theo_header = 'Theory = [\n'

#     groups = group_rows(prac='', date_from=date_patch())['rows']

#     # Удаляем старые частичные файлы, если нужно
#     for f in os.listdir(OUTPUT_DIR):
#         os.remove(os.path.join(OUTPUT_DIR, f))

#     # Параллельно обрабатываем каждую группу
#     with ThreadPoolExecutor(max_workers=10) as executor:
#         futures = [(executor.submit(process_and_save_group, group)) for group in groups]

#         for future in futures:
#             future.result()  # Дожидаемся выполнения, можно добавить обработку ошибок

#     # Собираем все результаты из файлов
#     practice_lines = ''
#     theory_lines = ''

#     for filename in os.listdir(OUTPUT_DIR):
#         with open(os.path.join(OUTPUT_DIR, filename), 'r', encoding='utf-8') as f:
#             data = json.load(f)
#             practice_lines += data.get('prac', '')
#             theory_lines += data.get('theo', '')

#     full_practice = prac_header + practice_lines + ']\n'
#     full_theory = theo_header + theory_lines + ']\n'

#     # Сохраняем финальный результат
#     with open(FINAL_FILE, 'w', encoding='utf-8') as f:
#         f.write(full_practice)
#         f.write(full_theory)

#     print("✅ Итоговой результат сохранён в", FINAL_FILE)
#     return [full_practice, full_theory]


# def process_and_save_group(group):
#     '''
#     Обработка одной группы и сохранение результата в JSON-файл
#     '''
#     group_id = group['id']
#     filename = os.path.join(OUTPUT_DIR, f"group_{group_id}.json")

#     # Проверяем, не обработана ли группа уже
#     if os.path.exists(filename):
#         print(f"🔁 Группа {group_id} уже обработана, пропускаем")
#         return

#     # Обрабатываем группу
#     prac_str, theo_str = process_group(group)

#     # Сохраняем результат
#     with open(filename, 'w', encoding='utf-8') as f:
#         json.dump({
#             'group_id': group_id,
#             'prac': prac_str,
#             'theo': theo_str
#         }, f, ensure_ascii=False, indent=2)

#     print(f"💾 Группа {group_id} сохранена")


In [41]:

def create_disc_list3():
    '''Создание списков Practice и Theory с сохранением промежуточных результатов'''
    groups = group_rows(prac='', date_from=date_patch())['rows']

    # Очистка предыдущих промежуточных файлов (по желанию)
    for f in os.listdir(OUTPUT_DIR):
        os.remove(os.path.join(OUTPUT_DIR, f))

    # Параллельная обработка групп
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(process_and_save_group, group) for group in groups]
        for future in futures:
            future.result()  # Ждём завершения задач

    # Сборка результата из всех промежуточных файлов
    practice_list = []
    theory_list = []
    current_id = 0

    for filename in os.listdir(OUTPUT_DIR):
        with open(os.path.join(OUTPUT_DIR, filename), 'r', encoding='utf-8') as f:
            data = json.load(f)

            for item in data.get("practice", []):
                item["id"] = current_id
                practice_list.append(item)
                current_id += 1

            for item in data.get("theory", []):
                item["id"] = current_id
                theory_list.append(item)
                current_id += 1

    # Финальная структура
    final_data = {
        "Practice": practice_list,
        "Theory": theory_list
    }

    # Сохраняем итоговой файл
    with open(FINAL_FILE, "w", encoding="utf-8") as f:
        json.dump(final_data, f, ensure_ascii=False, indent=2)

    print(f"✅ Результат успешно сохранён в {FINAL_FILE}")
    return final_data


def process_and_save_group(group):
    """
    Обрабатывает группу и сохраняет результат в JSON-файл
    """
    group_id = group["id"]
    filename = os.path.join(OUTPUT_DIR, f"group_{group_id}.json")

    if os.path.exists(filename):
        print(f"🔁 Группа {group_id} уже обработана, пропускаем")
        return

    print(f"🔄 Обработка группы {group_id}")

    practice_items, theory_items = process_group(group)

    # Сохранение промежуточного результата
    with open(filename, "w", encoding="utf-8") as f:
        json.dump({
            "group_id": group_id,
            "practice": practice_items,
            "theory": theory_items
        }, f, ensure_ascii=False, indent=2)

    print(f"💾 Группа {group_id} сохранена")


def process_group(group):
    """
    Обработка одной группы: сбор данных по предметам и студентам
    """
    name = group["name"].split(" ")[0] if " " in group["name"] else group["name"]
    group_id = group["id"]

    practice_items = []
    theory_items = []

    disc_data = disc_rows(group_id, prac='', date_from=date_patch())
    discs = disc_data.get('rows', [])

    for disc in discs:
        student_data = student_rows(group_id, disc['id'], prac='', date_from=date_patch())
        students = student_data.get('rows', [])

        if not students:
            continue
        print(1,disc)
        student_id = students[0]['student_id']
        full_name = f"{name}_{disc['name']}"

        # Предположим, что subject_id и sub_group_id есть в disc или где-то ещё
        subject_id = disc.get("id", 0)
        sub_group_id = disc.get("sub_group_id", 0)

        item = {
            "name": full_name,
            "id_group": group_id,
            "subject_id": subject_id,
            "sub_group_id": sub_group_id,
            "student_id": student_id,
            # "id" будет назначен позже глобально
        }

        if student_data['total'] <= 5:
            practice_items.append(item.copy())
            theory_items.append(item.copy())
        elif '(' in disc['name']:
            practice_items.append(item.copy())
        else:
            theory_items.append(item.copy())

    return practice_items, theory_items


In [ ]:
global login
if login(csrftoken, ssuz_sessionid):  # Если авторизация прошла успешно
  print("ok")
  # print(cookies)
  # print(head(cookies))

  # для просмотрат всех групп
  # group_rows(prac='', date_from=date_patch())

  # print(create_disc_list3())
create_disc_list3()



